# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../weatherpy/output_data/cities.csv")
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,savinka,50.08,47.10,25.09,72,0,9.91,RU,1605855250
1,pangnirtung,66.15,-65.71,6.31,88,47,3.78,CA,1605855250
2,hithadhoo,-0.60,73.08,81.59,69,100,5.68,MV,1605855251
3,kasane,-17.82,25.15,69.80,100,75,6.93,BW,1605855251
4,leninsk-kuznetskiy,54.66,86.17,15.80,78,90,11.18,RU,1605855251


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store lat and long in locations
locations = city_data[["Lat","Lng"]]

# Store humidity as weight
humidity = city_data["Humidity"].astype(float)

In [7]:
# Plot heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,max_intensity=humidity.max(),point_radius=3,dissipating=False)

# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
data_cleaned = city_data.dropna(how='any')

temp_filtered = data_cleaned[(data_cleaned["Max Temp"]<=80) & (data_cleaned["Max Temp"]>=65)]
humidity_filtered = temp_filtered[(temp_filtered["Humidity"]<=70) & (temp_filtered["Humidity"]>=50)]
wind_filtered = humidity_filtered[humidity_filtered["Wind Speed"]<5]
ideal_cities = wind_filtered[wind_filtered["Cloudiness"]<=10]
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,cape town,-33.93,18.42,71.01,52,0,3.36,ZA,1605855252
157,port macquarie,-31.43,152.92,79.00,60,0,1.01,AU,1605855266
307,odienne,9.51,-7.57,67.77,61,0,2.95,CI,1605855310
494,makakilo city,21.35,-158.09,78.80,69,1,4.70,US,1605855347
516,agaro,7.85,36.65,75.29,50,3,1.34,ET,1605855351
519,butembo,0.15,29.28,73.06,58,2,2.06,CD,1605855352


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Add Hotel Name column to df
ideal_cities["Hotel Name"] = ""
hotel_df = ideal_cities
hotel_df


C:\Users\novad\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,cape town,-33.93,18.42,71.01,52,0,3.36,ZA,1605855252,
157,port macquarie,-31.43,152.92,79.00,60,0,1.01,AU,1605855266,
307,odienne,9.51,-7.57,67.77,61,0,2.95,CI,1605855310,
494,makakilo city,21.35,-158.09,78.80,69,1,4.70,US,1605855347,
516,agaro,7.85,36.65,75.29,50,3,1.34,ET,1605855351,
519,butembo,0.15,29.28,73.06,58,2,2.06,CD,1605855352,


In [24]:
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# Set up parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    # get lat,lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    if row["Hotel Name"] == "":
        params["location"] = f"{lat},{lng}"
        
        # make request
        hotel_search = requests.get(base_url,params=params)
        
        # convert to json
        hotel_data = hotel_search.json()
        
        try:
            hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        except:
            hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"

hotel_df

C:\Users\novad\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,cape town,-33.93,18.42,71.01,52,0,3.36,ZA,1605855252,Blue Views Studios
157,port macquarie,-31.43,152.92,79.00,60,0,1.01,AU,1605855266,Sails Port Macquarie by Rydges
307,odienne,9.51,-7.57,67.77,61,0,2.95,CI,1605855310,HÔTEL SARAH
494,makakilo city,21.35,-158.09,78.80,69,1,4.70,US,1605855347,Marriott's Ko Olina Beach Club
516,agaro,7.85,36.65,75.29,50,3,1.34,ET,1605855351,No Hotel Found
519,butembo,0.15,29.28,73.06,58,2,2.06,CD,1605855352,Hotel Ivatsiro


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))